In [31]:
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
import numpy as np

In [32]:
# Chargement des données CIFAR-10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

In [33]:
# Prétraitement des images (normalisation)
train_images = (train_images.astype('float32') - 127.5) / 127.5
test_images = (test_images.astype('float32') - 127.5) / 127.5

In [34]:
# réseau de neurones convolutionnel (CNN)
cnn_input = layers.Input(shape=(32, 32, 3))
x = layers.Conv2D(32  , (3, 3), activation='relu')(cnn_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(2048, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(512, (3, 3), activation='relu')(x)
x = layers.Flatten()(x)
cnn_output = layers.Dense(256 , activation='relu')(x)

In [35]:
# Définition des deux branches du réseau siamois
siam_input_a = layers.Input(shape=(32, 32, 3))
siam_input_b = layers.Input(shape=(32, 32, 3))

In [36]:
# CNN sur le reseau siamois
siam_output_a = layers.Dense(64, activation='relu')(layers.Flatten()(layers.Conv2D(32, (3, 3), activation='relu')(siam_input_a)))
siam_output_b = layers.Dense(64, activation='relu')(layers.Flatten()(layers.Conv2D(32, (3, 3), activation='relu')(siam_input_b)))

In [37]:
# Concaténation des sorties des deux trucs
combined_features = layers.Concatenate()([siam_output_a, siam_output_b])

In [38]:
# Fusion des features
merged_features = layers.Concatenate()([cnn_output, combined_features])

In [39]:
# Couche pleinement connecté pour la classification
x = layers.Dense(64, activation='relu')(merged_features)
output = layers.Dense(10, activation='sigmoid')(x)


In [40]:
# Création du modèle
model = models.Model(inputs=[cnn_input, siam_input_a, siam_input_b], outputs=output)

In [41]:
# Compilation
model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [42]:
# Entraînement
#model.fit([train_images, train_images, train_images], train_labels, epochs=15, batch_size=128 , validation_data=([test_images, test_images, test_images], test_labels))

In [43]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Définition du chemin de sauvegarde du modèle
checkpoint_path = "best_classificator.keras"

# Définition du callback ModelCheckpoint pour sauvegarder le meilleur modèle
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             monitor='val_accuracy',  # Surveille la précision de validation
                             save_best_only=True,     # Ne sauvegarde que le meilleur modèle
                             verbose=1)               # Affiche des informations lors de la sauvegarde

# Entraînement du modèle avec le callback ModelCheckpoint
history = model.fit([train_images, train_images, train_images], train_labels,
                    epochs=7 , batch_size=128,
                    validation_data=([test_images, test_images, test_images], test_labels),
                    callbacks=[checkpoint])

# Affichage des résultats d'entraînement
print(history.history)

# Chargement du meilleur modèle sauvegardé
best_model = models.load_model(checkpoint_path)

# Évaluation du meilleur modèle sur les données de test
loss, accuracy = best_model.evaluate([test_images, test_images, test_images], test_labels)
print("Meilleur modèle - perte:", loss)
print("Meilleur modèle - précision:", accuracy)


Epoch 1/7
391/391 [==============================] - ETA: 0s - loss: 1.2992 - accuracy: 0.5374
Epoch 1: val_accuracy improved from -inf to 0.63400, saving model to best_classificator.keras
391/391 [==============================] - 35s 81ms/step - loss: 1.2992 - accuracy: 0.5374 - val_loss: 1.0473 - val_accuracy: 0.6340
Epoch 2/7
390/391 [============================>.] - ETA: 0s - loss: 0.8270 - accuracy: 0.7105
Epoch 2: val_accuracy improved from 0.63400 to 0.70460, saving model to best_classificator.keras
391/391 [==============================] - 28s 72ms/step - loss: 0.8273 - accuracy: 0.7104 - val_loss: 0.8546 - val_accuracy: 0.7046
Epoch 3/7
391/391 [==============================] - ETA: 0s - loss: 0.6094 - accuracy: 0.7885
Epoch 3: val_accuracy improved from 0.70460 to 0.75010, saving model to best_classificator.keras
391/391 [==============================] - 29s 75ms/step - loss: 0.6094 - accuracy: 0.7885 - val_loss: 0.7401 - val_accuracy: 0.7501
Epoch 4/7
391/391 [=========

In [44]:
'''from tensorflow.keras.callbacks import ModelCheckpoint

# Définir un rappel pour sauvegarder le meilleur modèle
checkpoint_callback = ModelCheckpoint(filepath='best_model.h5',
                                      monitor='val_accuracy',
                                      mode='max',
                                      save_best_only=True)

# Entraîner le modèle en utilisant le rappel
model.fit(train_images,train_images,train_images,
          validation_data=(test_images,test_labels),
          epochs=10,
          callbacks=[checkpoint_callback])'''


"from tensorflow.keras.callbacks import ModelCheckpoint\n\n# Définir un rappel pour sauvegarder le meilleur modèle\ncheckpoint_callback = ModelCheckpoint(filepath='best_model.h5',\n                                      monitor='val_accuracy',\n                                      mode='max',\n                                      save_best_only=True)\n\n# Entraîner le modèle en utilisant le rappel\nmodel.fit(train_images,train_images,train_images,\n          validation_data=(test_images,test_labels),\n          epochs=10,\n          callbacks=[checkpoint_callback])"

In [46]:
#best_model = models.load_model("best_classificator.keras" )